# Example Visualizations

This notebook provides example queries and visualizations.

## Setup
The cell below is used to 
* import required libraries
* setting up the connection to the Neo4j database
* define the D3 based HTML template for custom visualizations

In [ ]:
import pandas as pd 
import plotly.express as px
import pygal as pg
from string import Template
from IPython.core.display import display, HTML
from IPython.display import HTML, Javascript, display

neo4j_url=%env NEO4J_URL

%reload_ext cypher
%config CypherMagic.uri=neo4j_url + "/db/data"

def configure_d3():
    """Tell require where to get d3 from in `require(['d3'])`"""
    display(Javascript("""
    require.config({ 
      paths: {
        lodash: "/notebooks/vis/lib/lodash.min",  
        d3: "/notebooks/vis/lib/d3.v4.min"
      }
    })"""))

configure_d3()

base_html = """
<!DOCTYPE html>
<html>
  <head>
  <script type="text/javascript" src="/notebooks/vis/lib/svg.jquery.js"></script>
  <script type="text/javascript" src="/notebooks/vis/lib/pygal-tooltips.min.js""></script>
  </head>
  <body>
    <figure>
      {rendered_chart}
    </figure>
  </body>
</html>
"""

## Table
The simplest visualization is a table, the rows and columns are rendered directly from the result returned by the query.

In [ ]:
%cypher MATCH (a:Artifact)-[:CONTAINS]->(n:Type) RETURN a.fqn as Artifact, count(n) as TypesPerArtifact

## Pie Chart
A pie chart is used for illustrating proportions, e.g. package sizes. Therefore the query returns a row per package, each containing the name and the number of contained types.

In [ ]:
packageSize = %cypher MATCH (p:Package)-[:CONTAINS]->(t:Type) \
                      RETURN p.name as Package, count(DISTINCT t) AS Classes

df = packageSize.get_dataframe()
fig = px.pie(df, values='Classes', names='Package', title='Package Size')
fig.show()

## Circle Packing
A circle packing diagram can be used to illustrate hierarchical structures, e.g. packages and their children. The query returns a flattened tree structure containing one row per parent/child-combination with four columns:
* *Parent_Fqn*: the fully qualified name of the parent (e.g. type name including package name)
* *Parent_Name*: the name of the parent (e.g. type name without package name)
* *Child_Fqn*: the fully qualified name of the child
* *Child_Is_Leaf*: a boolean value that if true indicates that the child has no further children (e.g. true for a type, false for a package)

In [ ]:
packageHierarchy = %cypher MATCH (p:Package)-[:CONTAINS]->(t:Type) \
                           WITH DISTINCT p.fqn AS packageName \
                           MATCH (p:Package{fqn: packageName})-[:CONTAINS]->(child) \
                           WHERE (child:Package AND exists((child)-[:CONTAINS*]->(:Type))) OR child:Type \
                           WITH p, child, child:Type AS leaf \
                           RETURN DISTINCT p.fqn AS Parent_Fqn, p.name AS Parent_Name, child.fqn AS Child_Fqn, child.name AS Child_Name, leaf AS Child_Is_Leaf \
                           ORDER BY Parent_Fqn
packageHierarchy

In [ ]:
package_hierarchy_df = packageHierarchy.get_dataframe()
text = Template(open('../vis/circle-packing/circle-packing-diagram.html', 'r').read().replace("\n","")).substitute({
    'circle_data': package_hierarchy_df.to_csv(index = False).replace("\r\n","\\n").replace("\n","\\n"),
    'container': 'type-packing-diagram'
})

HTML(text)

## Chord Diagram
A chord diagram is used to illustrate dependencies between elements, e.g. packages. The query for each dependency returns
* *Source*: The name of the dependent element (e.g. source package)
* *Target*: The name of the element's dependency (e.g. target package)
* *X_Count*: The weight of of the dependency (e.g. the coupling between both packages)

In [ ]:
packageDependencies = %cypher MATCH (p1:Package)-[:CONTAINS]->(t1:Type), \
                                    (p2:Package)-[:CONTAINS]->(t2:Type), \
                                    (t1)-[dep:DEPENDS_ON]->(t2) \
                             WHERE  p1 <> p2 \
                             RETURN p1.name AS Source, \
                                    p2.name AS Target, \
                                    COUNT(dep) AS X_Count \
                             ORDER BY Source, Target

packageDependencies

In [ ]:
packageDependenciesData = packageDependencies.get_dataframe().to_csv(index = False).replace("\r\n","\\n").replace("\n","\\n")
text = Template(open('../vis/chord/chord-diagram.html', 'r').read().replace("\n","")).substitute({
    'chord_data': packageDependenciesData,
    'container': 'module-chord-diagram'})

HTML(text)